In [270]:
from manager import load_obj
import os
import re
import string
from collections import Counter
from pprint import pprint
from tqdm.notebook import tqdm
from itertools import combinations

In [271]:
works = load_obj("cleaned_corpus_topics")

In [272]:
count= 0
for work in works:
    if work.references != "none":
        count += 1

In [273]:
dist={}
for work in works:
    try:
        if work.topic != None:
            if work.topic not in dist:
                dist[work.topic] = 1
            else:
                dist[work.topic] += 1
    except AttributeError:
        continue

for k, v in dist.items():
    dist[k] = round(v/len(works), 4)

In [274]:
dist = dict(sorted(dist.items(), key=lambda item: item[1], reverse = True))
dist

{'Sprachwissenschaft': 0.1594,
 'Narratologie': 0.1295,
 'Familien, Geschlechter und deren Konflikte': 0.1076,
 'Politische Systeme, Kriegs- und Nachkriegsliteratur': 0.1056,
 'Philosophie': 0.0976,
 'Mittelalter': 0.0956,
 'Lyrik': 0.0956,
 'Untersuchung historischer Quellen (besonders religiöse)': 0.0657,
 'Theater, Drama und Tragödie': 0.0637,
 'Kommunikationswissenschaft': 0.0498,
 'Auseinandersetzungen mit dem Schul- und Lehrsystem': 0.0299}

In [275]:
index_topic ={}
for work in works:
    index_topic[work.link] = work.topic

In [276]:
index_topic

{'https://www.hausarbeiten.de/document/889420': 'Kommunikationswissenschaft',
 'https://www.hausarbeiten.de/document/1180614': 'Philosophie',
 'https://www.hausarbeiten.de/document/1035784': 'Narratologie',
 'https://www.hausarbeiten.de/document/1005442': 'Theater, Drama und Tragödie',
 'https://www.hausarbeiten.de/document/982747': 'Familien, Geschlechter und deren Konflikte',
 'https://www.hausarbeiten.de/document/976525': 'Mittelalter',
 'https://www.hausarbeiten.de/document/542604': 'Narratologie',
 'https://www.hausarbeiten.de/document/518472': 'Lyrik',
 'https://www.hausarbeiten.de/document/501982': 'Narratologie',
 'https://www.hausarbeiten.de/document/503415': 'Philosophie',
 'https://www.hausarbeiten.de/document/494152': 'Narratologie',
 'https://www.hausarbeiten.de/document/438018': 'Narratologie',
 'https://www.hausarbeiten.de/document/428153': 'Philosophie',
 'https://www.hausarbeiten.de/document/418214': 'Auseinandersetzungen mit dem Schul- und Lehrsystem',
 'https://www.h

In [277]:
index_work = {}
for index,work in enumerate(works):
    index_work[work.link.split("/")[-1]] = index

In [278]:
def find_tag(text, tag):
    
    reg_start = re.compile(fr"(\<{tag}\>)")
    reg_end =  re.compile(fr"(\<\/{tag}\>)")
    start = [m.start() for m in re.finditer(reg_start, text)]
    end = [m.start() for m in re.finditer(reg_end, text)]
    
    aut=""
    if start and end:
        for s,e in zip(start,end):
            aut += re.sub(r"<[^>]*>", "", text[s:e] + "  ")
    else:
        aut = ""
    return aut

In [279]:
def create_ref_dic(text):
    references ={"autoren": []}
    refs = text.split("<\/sequence")
    for ref in refs:
        text = find_tag(ref, "author")
        authors = re.split("  |\/", text)
        print(authors)
        authors = [x for x in authors if len(x) > 0]
        if len(authors) != 0:
            references["autoren"] += authors
    return references

# Get Most Common Topic per Author



In [280]:
def is_author(author, compare):
      """
        Arguments:
            author: a string of an author name
            compare: a string of another author name to compare with
        -checks if two author names are the same (e.g. Friedrich W. Nietzsche, F. Nietzsche)
        Returns:
            True if names are equal, False if names are not equal
           
    """
    name = author.split(" ")
    name = list(filter(None, name))

    name_compare = compare.split(" ")
    name_compare = list(filter(None, name_compare))
    flag = False
    full_name_flag = False
    if len(name) >= len(name_compare):
        prob = len(name_compare)
        tmp = name_compare
        name_compare = name
        name = tmp
    else:
        prob = len(name)
    new_prob = 0   

    beginners_n = [x[0] for x in name]
    beginners_nc = [x[0] for x in name_compare]
    
    
    
    for n in beginners_n:
        if n in beginners_nc:
            beginners_nc.remove(n)
        else:
            return False
            
    for i, n in enumerate(name):
            
            for j,nc in enumerate(name_compare):
                if (i==j) and (nc == n):
                    new_prob +=1
            
            if new_prob/prob > 0.5:
                return True
            
            elif n[0] not in (x[0] for x in name_compare):
                flag = False
                break

                
            elif (len(n) > 1) and n in name_compare:
                flag = True
                full_name_flag = True
    
            else:
              
                for nc in name_compare:
                    if (n[0] == nc[0]) and ((len(nc) == 1) or (len(n) == 1)):
                        flag = True    
                        
                    elif(n[0] == nc[0]):
                        control = min(len(n), len(nc))
                        if n[:control]!= nc[:control]:
                            return False
                    
                    
    if flag and full_name_flag:
        return True
    else:
        return False

In [281]:
with open('names_with_work.txt', encoding= "utf-8") as f:
    lines = [line.rstrip() for line in f]

In [282]:
aut_dic ={}
for line in lines:
    names = line.split("\t")
    aut_dic[names[0]] = names[1:]

In [283]:
single_aut={}
for key, value in tqdm(aut_dic.items()):
    auts = [x.translate(str.maketrans('', '', string.punctuation)) for x in value]
    
    if len(value) > 1:
        single_aut[key]=[auts[0]] 
    
    
        for name in auts[1:]:
            flag= False
            for n in single_aut[key]:
                if is_author(name, n):
                    flag = True
            if not flag:
                single_aut[key].append(name)

    elif len(value) == 1:
        
        single_aut[key]=[auts[0]]
        
    else:
        continue

  0%|          | 0/502 [00:00<?, ?it/s]

In [284]:
result ={}
for key, value in tqdm(single_aut.items()):
    compare = list(result)
    
    for name in value:
        flag = True
        for n in compare:
            if is_author(name,n):
                result[n].append([name, index_topic[key]])
                flag=False
                break
            else:
                continue
        if flag:
            result[name]=[[name,index_topic[key]]]
        
    
    

  0%|          | 0/488 [00:00<?, ?it/s]

In [285]:
result = dict(sorted(result.items(), key=lambda item: len(item[1]), reverse = True))

In [286]:
print(result)

{'goethe johann wolfgang': [['goethe johann wolfgang', 'Theater, Drama und Tragödie'], ['johann wolfgang', 'Philosophie'], ['goethe johann wolfgang', 'Narratologie'], ['goethe johann', 'Philosophie'], ['goethe johann wolfgang', 'Familien, Geschlechter und deren Konflikte'], ['goethe johann wolfgang', 'Narratologie'], ['goethe johann wolfgang', 'Familien, Geschlechter und deren Konflikte'], ['goethe johann wolfgang', 'Narratologie'], ['johann wolfgang von goethe', 'Philosophie'], ['johann wolfgang goethe', 'Theater, Drama und Tragödie'], ['goethe johann', 'Familien, Geschlechter und deren Konflikte'], ['johann wolfgang goethe', 'Lyrik'], ['goethe johann wolfgang', 'Lyrik'], ['goethe johann wolfgang', 'Familien, Geschlechter und deren Konflikte'], ['johann wolfgang von goethe', 'Politische Systeme, Kriegs- und Nachkriegsliteratur'], ['goethe johann wolfgang', 'Familien, Geschlechter und deren Konflikte'], ['goethe johann', 'Familien, Geschlechter und deren Konflikte'], ['goethe johann', 

In [1]:
#get the most dominant topic of an author

from collections import Counter
dominant_topic={}
for key, value in result.items():
    topics = dict(Counter([x[1] for x in value]))
    highest = max(topics, key=topics.get)
    dominant_topic[key] = highest


NameError: name 'result' is not defined

In [288]:
dominant_topic

{'goethe johann wolfgang': 'Familien, Geschlechter und deren Konflikte',
 'bumke joachim': 'Mittelalter',
 'dürrenmatt friedrich': 'Theater, Drama und Tragödie',
 'lessing gotthold ephraim': 'Theater, Drama und Tragödie',
 'schiller friedrich': 'Familien, Geschlechter und deren Konflikte',
 'mann thomas „der': 'Familien, Geschlechter und deren Konflikte',
 'nietzsche friedrich': 'Narratologie',
 'freud sigmund': 'Philosophie',
 'hoffmann eta': 'Narratologie',
 'adorno theodor w': 'Lyrik',
 'bußmann hadumod': 'Sprachwissenschaft',
 'hartmann von': 'Mittelalter',
 'wilpert gero': 'Narratologie',
 'fleischer wolfgang': 'Sprachwissenschaft',
 'franz kafka': 'Politische Systeme, Kriegs- und Nachkriegsliteratur',
 'martinez matias': 'Narratologie',
 'grimm jacob': 'Mittelalter',
 'büchner georg': 'Narratologie',
 'arnold heinz': 'Theater, Drama und Tragödie',
 'ruh kurt': 'Mittelalter',
 'weddige hilkert': 'Mittelalter',
 'heinzle joachim': 'Mittelalter',
 'wolfram von eschenbach': 'Mittelal

In [289]:
final_result = {}
for key, value in result.items():
    final_result[key] = len(value)

In [290]:
final_result

{'goethe johann wolfgang': 33,
 'bumke joachim': 22,
 'dürrenmatt friedrich': 11,
 'lessing gotthold ephraim': 11,
 'schiller friedrich': 10,
 'mann thomas „der': 10,
 'nietzsche friedrich': 9,
 'freud sigmund': 8,
 'hoffmann eta': 8,
 'adorno theodor w': 8,
 'bußmann hadumod': 8,
 'hartmann von': 8,
 'wilpert gero': 8,
 'fleischer wolfgang': 7,
 'franz kafka': 7,
 'martinez matias': 7,
 'grimm jacob': 7,
 'büchner georg': 6,
 'arnold heinz': 6,
 'ruh kurt': 6,
 'weddige hilkert': 6,
 'heinzle joachim': 6,
 'wolfram von eschenbach': 6,
 'foucault michel': 6,
 'brecht bertolt': 6,
 'bachmann ingeborg': 6,
 'polenz von peter': 6,
 'alt peterandr': 5,
 'meier albert': 5,
 'knopf jan friedrich': 5,
 'martini fritz': 5,
 'hermand jost': 5,
 'celan paul': 5,
 'gadamer hansgeorg': 5,
 'kant immanuel': 5,
 'eco umberto': 5,
 'haug walter': 5,
 'lewandowski theodor': 5,
 'scheffel michael': 5,
 'neumann gerhard': 5,
 'schröder werner': 5,
 'emmerich wolfgang': 5,
 'wehrli max': 5,
 'jost schnei

# Find Co-authors and topic from author


In [291]:
two_occurences = [k for k,v in final_result.items() if v>=2]

In [292]:
#remove some names manually

two_occurences.remove("h henne")
two_occurences.remove("hartweg frédéric wegera klauspeter") #not entirely sure which name
two_occurences.remove("e t a") #too many variations
two_occurences.remove("la fontaines“")
two_occurences.remove("durchges aufl")

In [293]:

for index,k in enumerate(two_occurences):
    if k == "mann thomas „der":
        two_occurences[index] = "mann thomas"
    if k == "b aumgart reinhard":
        two_occurences[index] = "baumgart reinhard"
    if k == "knopf jan friedrich":
        two_occurences[index] = "knopf jan"
    if k == "karthaus ulrich sturm":
        two_occurences[index] = "karthaus ulrich"
    if k == "blessin stefan johann wolfgang":
        two_occurences[index] = "blessin stefan"
    if k == "kracht christian faserland":
        two_occurences[index] = "kracht christian"
    if k == "hartmann von":
        two_occurences[index] = "hartmann von aue"

In [294]:
two_occurences

['goethe johann wolfgang',
 'bumke joachim',
 'dürrenmatt friedrich',
 'lessing gotthold ephraim',
 'schiller friedrich',
 'mann thomas',
 'nietzsche friedrich',
 'freud sigmund',
 'hoffmann eta',
 'adorno theodor w',
 'bußmann hadumod',
 'hartmann von aue',
 'wilpert gero',
 'fleischer wolfgang',
 'franz kafka',
 'martinez matias',
 'grimm jacob',
 'büchner georg',
 'arnold heinz',
 'ruh kurt',
 'weddige hilkert',
 'heinzle joachim',
 'wolfram von eschenbach',
 'foucault michel',
 'brecht bertolt',
 'bachmann ingeborg',
 'polenz von peter',
 'alt peterandr',
 'meier albert',
 'knopf jan',
 'martini fritz',
 'hermand jost',
 'celan paul',
 'gadamer hansgeorg',
 'kant immanuel',
 'eco umberto',
 'haug walter',
 'lewandowski theodor',
 'scheffel michael',
 'neumann gerhard',
 'schröder werner',
 'emmerich wolfgang',
 'wehrli max',
 'jost schneider',
 'kleist heinrich von',
 'schmidt jochen',
 'frenzel elisabeth',
 'schnell rüdiger',
 'saussure ferdinand',
 'wiese benno',
 'benn gottfried

In [295]:
single_aut.pop("https://www.hausarbeiten.de/document/99725") ## because of duplicate work


['aust hugo',
 'doering sabine',
 'gnam andrea',
 'grathoff dirk',
 'gönner gerhard',
 'vinken barbara',
 'kriegsfall ',
 'heinrich von kleist',
 'neumann gerhard',
 'sembdner helmut',
 'stephens anthony']

In [296]:
single_aut

{'https://www.hausarbeiten.de/document/889420': ['barner wilfried',
  'brake michael lars',
  'glanz berit',
  'glaser peter',
  'hertwig johannes',
  'schlegel friedrich',
  'schmitzemans monika „der'],
 'https://www.hausarbeiten.de/document/1180614': ['schiller friedrich',
  'alt peterandr',
  'luserkejaqui matthias',
  'middel carina',
  'nilges yvonne',
  'schmidhäuser eberhard',
  'borgards roland',
  'martus steffen',
  'vogl joseph',
  'mazza ethel',
  'willems gottfried'],
 'https://www.hausarbeiten.de/document/1035784': ['lovecraft hp',
  'stoker bram',
  'amann wilhelm',
  'frenschkowski marco',
  'freyermuth gundolf s',
  'glaser frank',
  'günzel stephan',
  'hanke christiane',
  'jones mark',
  'kreienbrink mattias',
  'mai anna',
  'mersch dieter',
  'murray chris',
  'neeb christian',
  'neubauer paul',
  'pacher jörg',
  'peak david',
  'pranz sebastian',
  'sanin andrey',
  'schell jesse',
  'smith don g hp',
  'sprödefeld paula',
  'wilson eric',
  'zaidan sarah pilbe

In [297]:
# get the co-authorship of each author
cowriters = {}
for elem in two_occurences:
    cowriters[elem] = []


for work in tqdm(works):
    auts = single_aut.get(work.link)
    compare_list = list(cowriters)
    
    possible =[]
    if auts:
        #check for people who occur at least twice in the works
        for aut in auts:
            for n in compare_list:
                if is_author(aut, n):
                    possible.append(n) ##possible error occurence, because here the name variation will be changed to a general one
                    
                    continue
        print(possible)
        print(auts)
        print("\n")
        
        for index,person in enumerate(possible):
            others =  [x for i,x in enumerate(possible) if i!=index]
            
            for p in compare_list:    
                if is_author(person,p):
                    cowriters[p] += others
        
        
    else:
        continue
  

    

  0%|          | 0/502 [00:00<?, ?it/s]

['barner wilfried', 'schmitzemans monika „der']
['barner wilfried', 'brake michael lars', 'glanz berit', 'glaser peter', 'hertwig johannes', 'schlegel friedrich', 'schmitzemans monika „der']


['schiller friedrich', 'alt peterandr', 'luserkejaqui matthias', 'willems gottfried']
['schiller friedrich', 'alt peterandr', 'luserkejaqui matthias', 'middel carina', 'nilges yvonne', 'schmidhäuser eberhard', 'borgards roland', 'martus steffen', 'vogl joseph', 'mazza ethel', 'willems gottfried']


['stoker bram', 'smith don g hp']
['lovecraft hp', 'stoker bram', 'amann wilhelm', 'frenschkowski marco', 'freyermuth gundolf s', 'glaser frank', 'günzel stephan', 'hanke christiane', 'jones mark', 'kreienbrink mattias', 'mai anna', 'mersch dieter', 'murray chris', 'neeb christian', 'neubauer paul', 'pacher jörg', 'peak david', 'pranz sebastian', 'sanin andrey', 'schell jesse', 'smith don g hp', 'sprödefeld paula', 'wilson eric', 'zaidan sarah pilbeam richard', 'dark souls ii', 'bandai namco games', 'd

['althoff gerd', 'bumke joachim', 'eco umberto', 'gadamer hansgeorg', 'haug walter', 'heinzle joachim', 'jaspers karl', 'müller jandirk', 'nagel bert', 'schulze ursula', 'schmidtwiegand ruth', 'wolfgang stammler', 'steinecke horst wenzel']
['althoff gerd', 'bumke joachim', 'petersen stefan', 'eco umberto', 'gadamer hansgeorg', 'haug walter', 'heinzle joachim', 'jönsson maren', 'leyser karl j', 'müller jandirk', 'müller gernot', 'nagel bert', 'schreiner klaus', 'metaphorik kommunikative', 'repräsentation das', 'schulze ursula', 'philipowski silke', 'schmidtwiegand ruth', 'wolfgang stammler', 'wandhoff heiko', 'steger hartmut', 'steinecke horst wenzel']


['braune wilhelm', 'heather peter j', 'lewandowski theodor', 'weddige hilkert']
['bierbrauer volker', 'braune wilhelm', 'heidermanns frank', 'heather peter j', 'könig werner', 'krause wolfgang', 'lewandowski theodor', 'streitberg wilhelm', 'szemerenyi oswald', 'weddige hilkert']


['bernhard thomas', 'goethe johann wolfgang', 'franz kaf

['karlotto conrady', 'kommerell max', 'korff hermann a', 'staiger emil']
['beutler ernst peter', 'christ kurt jacobi', 'conrady karl', 'dietze walter', 'dünzer heinrich', 'grenzmann wilhelm goethe', 'haile harry g', 'hertel  das', 'kommerell max', 'korff hermann a', 'meinecke friedrich lebenströster', 'metzlerliteraturlexikon begriffe', 'rosenberg rainer', 'schaefer johann', 'wilhelm goethe’s', 'staiger emil', 'titzmann michael vom', 'vollmer wilhelm']


['gottfried keller', 'kaiser gerhard', 'müller christian', 'pfotenhauer helmut', 'richter hans']
['gottfried keller', 'kaiser gerhard', 'lenze franz', 'jahr ag', 'mccord norman', 'mohaupt dr lutz', 'müller christian', 'neumann bernd', 'pfotenhauer helmut', 'richter hans', 'talgeri pramod']


['peter gallmann']
['carstensen broder', 'fink hermann', 'gallmann peter', 'gregor bernd', 'grimm hansjürgen gertraud', 'hennig john', 'jablonski miroslaw', 'jung walter', 'lamprecht adolf', 'sachs emmy', 'schultebeckhausen marion', 'sick bastian',

['goethe johann wolfgang', 'mann thomas', 'brüggemann heinz peter', 'freund winfried', 'plumpe gerhard', 'renner rolf', 'wiese benno', 'wilpert gero']
['goethe johann wolfgang', 'mann thomas', 'brüggemann heinz peter', 'freund winfried', 'kuzniar alice a', 'plumpe gerhard', 'renner rolf', 'wiese benno', 'wilpert gero']


['nietzsche friedrich', 'wedekind frank']
['nietzsche friedrich', 'wedekind frank', 'brummarck jürgen']


['wolfram von eschenbach', 'chrétien de troyes', 'brüder grimm', 'bumke joachim', 'dewald hans', 'johnson l', 'köhler erich']
['wolfram von eschenbach', 'chrétien de troyes', 'brüder grimm', 'bumke joachim', 'dewald hans', 'johnson l', 'köhler erich', 'möhren markus', 'schirok bernd']


['carl johann', 'harvey william', 'hoffmann friedrich', 'mauser wolfram', 'uz johann peter']
['bergdolt klaus', 'breitinger johann jacob', 'bodmer johann jacob', 'carl johann', 'egger irmtraud', 'glauer reiner', 'harvey william', 'hoffmann friedrich', 'hufeland christoph wilhelm', '

...

['wolf g']
['bobrowski j', 'selbstzeugnisse gesammelte', 'gajek b', 'haufe e', 'wolf g', 'orlowski h', 'lustner b', 'meckel ch']


['kraft herbert']
['huge walter', 'kraft herbert', 'schneider ronald', 'hülshoff j b']


['weinhold ulrike', 'allerdissen rolf arthur']
['weinhold ulrike', 'aurnhammer achim', 'allerdissen rolf', 'boner georgette', 'schmidtdengler wendelin', 'reihe a']


['thomas anz', 'frenzel elisabeth']
['abraham ulf', 'anz thomas franz', 'frenzel elisabeth', 'petr pavel', 'wagenbach klaus']


['henne helmut', 'friedrich hugo', 'friedrich hansedwin', 'hölderlin friedrich', 'hoffmann friedrich']
['henne helmut', 'schulze dr', 'tenbruck friedrich h', 'turner r steven', 'universitäten handbuch']


['goethe johann wolfgang', 'martini fritz', 'metzler literaturlexikon']
['goethe johann wolfgang', 'martini fritz', 'metzler literaturlexikon', 'preisendanz wolfgang', 'stauch von', 'quitzow wolfgang']


['pinthus kurt']
['eykman christoph', 'pinthus kurt']


['brecht bertolt', 'b

['derrida jacques']
['derrida jacques', 'lévistrauss claude', 'bolz norbert']


['bußmann hadumod', 'derrida jacques', 'hempfer k', 'saussure ferdinand', 'vigener gerhard']
['bußmann hadumod', 'derrida jacques', 'houdebine guy', 'hempfer klaus', 'klein josef', 'maak niklas', 'menke bettina', 'de saussure ferdinand', 'vigener gerhard']


['olsen s']
['ibsen henrik', 'andreassalomé lou henrik', 'ibsens frauengestalten jena', 'durbach errol', 'knoben marianne', 'lingard lorelei', 'mayerson caroline w', 'mcfarlane james ibsen', 'olsen stein', 'panse irischarlotte', 'raphael robert', 'saari sandra e', 'shideler ross', 'carsten brettschneider']


['kleist heinrich von', 'horn peter', 'schröder jürgen']
['kleist heinrich', 'hoffmeister werner', 'horn peter', 'marx stefanie', 'schröder jürgen', 'sennett richard']


[]
['beutner e', 'laemmle p', 'pfabigan a', 'schmidtthieme h', 'einzeldarstellungen volk']


['bachtin michail', 'hermann broch', 'ellmann richardjames', 'kristeva julia']
['james j


['ellmann richardjames', 'joyce james', 'zimmer michael']
['ellmann richard james', 'james joyce', 'helmling steven', 'king john', 'paris jean', 'saunders max', 'zimmermann michael']


[]
['the europeans gb', 'booth wayne c']


['dürrenmatt friedrich']
['dürrenmatt friedrich', 'grosse wilhelm', 'wagner hans']


['bumke joachim', 'de boor helmut', 'ehlert trude', 'einhorn jürgen', 'gerd heinzmohr', 'maurer friedrich', 'ruh kurt', 'schnell rüdiger', 'schöpf hans', 'weddige hilkert']
['bumke joachim', 'brummak jürgen', 'de boor helmut', 'ders h', 'ehlert trude', 'dies der', 'einhorn jürgen w', 'grammel elisabeth', 'haupt barbara', 'lenz friedrich', 'mohr gerd heinz', 'maurer friedrich', 'offermanns winfried', 'ruh kurt', 'schnell rüdiger', 'schöpf hans', 'stackmann karl', 'szklenar hans', 'urbanek friedrich', 'weddige hilkert', 'wilmanns w alexander']


['s uhrbier hartwig', 'wiese benno']
['eichendorff joseph', 'wolfgang reichmann', 'haar carel', 'schultz hartwig', 'wiese benno']


['go

['honcamp franz c', 'schnitzler arthur']
['coudenhouve ', 'kalergi richard', 'csokor franz', 'dollinger petra', 'dubrovic milan', 'heydenrynsch verena', 'meysels lucian o', 'redl renate', 'salm graf', 'schnitzler arthur', 'seibert peter', 'roberto turk', 'szepszuckerkandl berta', 'veigl hans', 'wilhelmydollinger petra', 'zuckerkandl bertha']


['schönbach anton', 'brinkmann hennig der', 'ruh kurt', 'paul hermann']
['mone franz', 'pfeiffer franz', 'schönbach anton', 'wackernagel wilhelm', 'morvay karin', 'grube dagmar', 'ladischgrube dagmar', 'cruel rudolf', 'löffler karl', 'brinkmann hennig', 'frey winfried', 'lindner pirmin', 'linsenmayer anton', 'rengstorf karl', 'rost hans', 'ruh kurt', 'hennig beate', 'paul hermann', 'neue jerusalemer']


['heinrich mann', 'mann klaus', 'mann thomas']
['lion ferdinand', 'mann heinrich', 'mann klaus', 'mann thomas', 'pannwitz rudolf', 'schickele rené']


['wulf joseph', 'brecht carl schmitt']
['bendersky joseph w', 'bohrer karl', 'schmitt carl', 'so

['wehrli max']
['vogt friedrich', 'ehrismann gustav', 'wehrli max']


['george stefan', 'nietzsche friedrich', 'rasch wolfdietrich', 'kafitz dieter', 'claude david stefan', ' hans joachim', 'jens malte fischer', 'hermand jost']
['stefan george', 'georg simmel', 'friedrich nietzsche der', 'joriskarl huysmans', 'christa bürger', 'wolfdietrich rasch', 'günter heintz', 'volkmar hansen', 'rudolf walter', 'dieter kafitz', 'karlhans kluncker', 'michael worbs', 'claude david', 'hans joachim piechotta', 'jens malte fischer', 'hannes böhringer', 'peter sprengler', 'richard hamann', 'reinhard farkas', 'jost hermand', 'erwin koppen', 'jürgen schutte', 'erich ruprecht', 'hanspeter bayerdörfer', 'arnold hauser']


['von la roche']
['haller michael', 'von la', 'roche walther']


['bienek horst']
['bienek horst', 'jens walter', 'mecklenburg norbert', 'chwin stefan']


['brant sebastian das', 'barbara könneker']
['brant sebastian', 'könneker barbara', 'mezger werner', 'gaier ulrich']


['korff hermann 

In [298]:
#show how many time authors cited together
for key,value in cowriters.items():
    new_dic={}
    for name in value:
        if name not in new_dic:
            new_dic[name] = 1
        else:
            new_dic[name] +=1

    new_dic =dict(sorted(new_dic.items(), key=lambda item: item[1], reverse = True))

    cowriters[key] = new_dic

In [299]:
cowriters = dict(sorted(cowriters.items(), key=lambda item: len(item[1]), reverse = True))

In [300]:
cowriters

{'goethe johann wolfgang': {'nietzsche friedrich': 3,
  'eckermann johann peter': 3,
  'wilpert gero': 3,
  'müller joachim': 3,
  'meier albert': 2,
  'rousseau jeanjacques': 2,
  'neumann gerhard': 2,
  'mann thomas': 2,
  'freund winfried': 2,
  'blessin stefan': 2,
  'karthaus ulrich': 2,
  'vaget hans rudolf': 2,
  'martini fritz': 2,
  'hamm heinz': 2,
  'schmidt jochen': 2,
  'george stefan': 2,
  'herder johann': 1,
  'bernhard thomas': 1,
  'franz kafka': 1,
  'schiller friedrich': 1,
  'stifter adalbert': 1,
  'walser robert': 1,
  'albes claudia': 1,
  'fischer bernhard': 1,
  'kurscheidt georg': 1,
  'rothemann sabine': 1,
  'schelle karl': 1,
  'michel georg': 1,
  'pfotenhauer helmut': 1,
  'brüggemann heinz peter': 1,
  'plumpe gerhard': 1,
  'renner rolf': 1,
  'wiese benno': 1,
  'lessing gotthold ephraim': 1,
  'pütz peter': 1,
  'luserkejaqui matthias': 1,
  'karl viëtor': 1,
  'wolfgang kayser': 1,
  'haug walter': 1,
  'walter huder': 1,
  'kommerell max': 1,
  'se

In [301]:
delete=[]
for key, value in cowriters.items():
    new={}
    for k,v in value.items():
        if v>1:
            new[k]=v
    if len(new) ==0:
        delete.append(key)
    cowriters[key] = new
for p in delete:
    cowriters.pop(p)

In [302]:
cowriters

{'goethe johann wolfgang': {'nietzsche friedrich': 3,
  'eckermann johann peter': 3,
  'wilpert gero': 3,
  'müller joachim': 3,
  'meier albert': 2,
  'rousseau jeanjacques': 2,
  'neumann gerhard': 2,
  'mann thomas': 2,
  'freund winfried': 2,
  'blessin stefan': 2,
  'karthaus ulrich': 2,
  'vaget hans rudolf': 2,
  'martini fritz': 2,
  'hamm heinz': 2,
  'schmidt jochen': 2,
  'george stefan': 2},
 'bumke joachim': {'wolfram von eschenbach': 4,
  'hartmann von aue': 4,
  'weddige hilkert': 3,
  'schnell rüdiger': 3,
  'wapnewski peter': 3,
  'cormeau christoph': 2,
  'ruh kurt': 2,
  'althoff gerd': 2,
  'eco umberto': 2,
  'haug walter': 2,
  'heinzle joachim': 2,
  'schröder werner': 2,
  'chrétien de troyes': 2,
  'johnson l': 2,
  'keen maurice': 2,
  'duby georges': 2,
  'ranke friedrich': 2,
  'wehrli max': 2,
  'reichert hermann': 2,
  'maurer friedrich': 2},
 'nietzsche friedrich': {'goethe johann wolfgang': 3,
  'walser robert': 2,
  'albes claudia': 2,
  'kurscheidt geo

In [304]:
id_dic={}
for index, k in enumerate(cowriters.keys()):
    id_dic[k] = index

In [305]:
len(id_dic)

186

# Create nodes and Edgelist for graph

In [306]:
#### create node csv
import csv

with open("gephi/node_germanistik_topic.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Id", "Label", "color VARCHAR"])
    for key, value in id_dic.items():
        names = dominant_topic.keys()
        for n in names:
            if is_author(key,n):
                writer.writerow([value, key, dominant_topic.get(n)])
                break

In [307]:
### create edge csv
import csv

with open("gephi/edge_germanisitk_topic.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Source", "Target", "Weight"])
    for key, value in cowriters.items():
        for k,v in value.items():
            writer.writerow([id_dic.get(key), id_dic.get(k), v])